In [16]:
import numpy as np
import pandas as pd

In [17]:
df=pd.read_csv('diabetes.csv')

In [18]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [19]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [20]:
x= df.iloc[:,:-1].values
y=df.iloc[:,-1]

In [21]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [22]:
x=scaler.fit_transform(x)

In [23]:
x.shape

(768, 8)

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [58]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [26]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

C:\Users\spatr\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
model.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.5170 - loss: 0.7036 - val_accuracy: 0.6688 - val_loss: 0.6491
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6430 - loss: 0.6540 - val_accuracy: 0.7208 - val_loss: 0.6013
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6869 - loss: 0.6053 - val_accuracy: 0.7727 - val_loss: 0.5664
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7187 - loss: 0.5668 - val_accuracy: 0.7597 - val_loss: 0.5423
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7225 - loss: 0.5471 - val_accuracy: 0.7532 - val_loss: 0.5239
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7431 - loss: 0.5368 - val_accuracy: 0.7532 - val_loss: 0.5101
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7571 - loss: 0.5079 - val_accuracy: 0.7597 - val_loss: 0.5000
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7428 - loss: 0.5182 - val_accuracy: 0.7532 - val_loss

In [28]:
#1 how to select appropriate optimizer
#2 No, of nodes in a layer
#3 how to select no of layers
#4 All in one model

In [29]:
pip install -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [33]:
# Install keras-tuner if not already installed
# !pip install -U keras-tuner

import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model building function
def build_model(hp):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    
    # Choose optimizer from the list
    optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop', 'adadelta'])
    
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Define the hyperparameter tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',  # Metric to optimize
    max_trials=5,              # Number of trials
)

tuner.search(x_train,y_train,epochs=10,validation_data=(x_test,y_test))


Reloading Tuner from .\untitled_project\tuner0.json


In [34]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

In [35]:
model=tuner.get_best_models(num_models=1)[0]

In [36]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=11,validation_data=(x_test,y_test))

Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6917 - loss: 0.5849 - val_accuracy: 0.7727 - val_loss: 0.5584
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7372 - loss: 0.5558 - val_accuracy: 0.7727 - val_loss: 0.5508
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7433 - loss: 0.5444 - val_accuracy: 0.7662 - val_loss: 0.5453
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7307 - loss: 0.5554 - val_accuracy: 0.7662 - val_loss: 0.5395
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7544 - loss: 0.5235 - val_accuracy: 0.7662 - val_loss: 0.5343
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7916 - loss: 0.5118 - val_accuracy: 0.7662 - val_loss: 0.5294
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7570 - loss: 0.5291 - val_accuracy: 0.7597 - val_loss: 0.5255
Epoch 19/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7316 - loss: 0.5468 - val_accuracy: 0

In [38]:
def build_model(hp):
    model=Sequential()
    units=hp.Int('units',min_value=8,max_value=128,step=8)
    model.add(Dense(units=units, activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [39]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,
                     directory='mydir')

Reloading Tuner from mydir\untitled_project\tuner0.json


In [40]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [41]:
tuner.get_best_hyperparameters()[0].values

{'units': 24}

In [42]:
model=tuner.get_best_models(num_models=1)[0]

In [43]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7502 - loss: 0.55572
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7699 - loss: 0.5424
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7547 - loss: 0.5344
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7641 - loss: 0.5238
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7634 - loss: 0.5102
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7573 - loss: 0.5119  
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7408 - loss: 0.5269  
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7431 - loss: 0.5102
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7501 - loss: 0.5057
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7585 - loss: 0.5043
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7636 - loss: 0.4882
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

In [44]:
def build_model(hp):
    model=Sequential()
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [45]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=3,
                     directory='my_dir',
                     project_name='num_layers')

Reloading Tuner from my_dir\num_layers\tuner0.json


In [46]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [47]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [48]:
model=tuner.get_best_models(num_models=1)[0]

C:\Users\spatr\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [49]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.7987 - loss: 0.4355 - val_accuracy: 0.7922 - val_loss: 0.4778
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7793 - loss: 0.4419 - val_accuracy: 0.7922 - val_loss: 0.4737
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8010 - loss: 0.4218 - val_accuracy: 0.7792 - val_loss: 0.4778
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8349 - loss: 0.3890 - val_accuracy: 0.8182 - val_loss: 0.4801
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7869 - loss: 0.4243 - val_accuracy: 0.7727 - val_loss: 0.5010
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8124 - loss: 0.4092 - val_accuracy: 0.8052 - val_loss: 0.4738
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7922 - loss: 0.4038 - val_accuracy: 0.7922 - val_loss: 0.4841
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8176 - loss: 0.4119 - val_accuracy: 0.74

In [59]:
def build_model(hp):
    model = Sequential()
    c = 0

    # Define the number of layers
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        if c == 0:
            # Add first layer with input dimension
            model.add(Dense(
                units=hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
                input_dim=8
            ))
            model.add(Dropout(hp.Choice('dropout' + str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            # Add subsequent layers
            model.add(Dense(
                units=hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
            ))
            model.add(Dropout(hp.Choice('dropout' + str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        c += 1

    # Add the output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(
        optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [61]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=3,
                     directory='my_dir',
                     project_name='final1')

In [62]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 08s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 17s


In [63]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 64,
 'activation0': 'relu',
 'dropout0': 0.3,
 'optimizer': 'adam'}

In [64]:
model=tuner.get_best_models(num_models=1)[0]


C:\Users\spatr\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [65]:
model.fit(x_train,y_train,epochs=200,initial_epoch=5,validation_data=(x_test,y_test))

Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7285 - loss: 0.5883 - val_accuracy: 0.7727 - val_loss: 0.5208
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7453 - loss: 0.5502 - val_accuracy: 0.7727 - val_loss: 0.5013
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7711 - loss: 0.4937 - val_accuracy: 0.7792 - val_loss: 0.4872
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7605 - loss: 0.5098 - val_accuracy: 0.7792 - val_loss: 0.4777
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7630 - loss: 0.4952 - val_accuracy: 0.7792 - val_loss: 0.4718
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7864 - loss: 0.4780 - val_accuracy: 0.7792 - val_loss: 0.4686
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7733 - loss: 0.4995 - val_accuracy: 0.7922 - val_loss: 0.4673
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7838 - loss: 0.4644 - val_accuracy: 0.785